In [1]:
%run "..\general_functions\Extracting Data Functions.ipynb"
%run "..\general_functions\generalFunctions.ipynb"

### Data Parameters

In [2]:
f_name = os.path.join(os.path.dirname(os.getcwd()),"Edgewell US Male Dataset.xlsx")
client_manuf = ["Edgewell Personal Care"]
# client_brands = ["Cremo","Xtreme 3","Hydro 5","Xtreme 2",
#                  "Quattro For Men","Hydro 3","Hydro Stubble Eraser",
#                  "Xtreme 4","Schick Classic","Schick Injector",
#                  "Schick Original","Xtreme 5","St2/Slim Twin",
#                  "5-Blade","The Stylish 5","Speed 3 Sensitive",
#                  "The Essential 2","The Stylish 3","Caliber 4 Max",
#                  "Easy Fit 3","Speed 4","Pbg Pl"]
 
client_brands = ["Schick","Equate","Cremo"]
decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

prodORitem="SKU"
categories =["Manual Shave Men"]
sectors = ["System","Disposables"]
segments = ["Razors","Refills","Disposables"]
subsegments= []
subcategories= []

customareas=''
national = True
areas = ["NATIONAL",'RETAILER']

regions_RET  =["Bj's And Sam's","Walmart"]
channels_RET = []
market_RET = []
 
regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []
 
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending March  2025"
end_date = "2025-04-01"
 
direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories)),
                "Segment":list(zip(["Sector"]*len(sectors),sectors)),
                "SubSegment":list(zip(["Segment"]*len(segments),segments)),
                "SubCategory":list(zip(["Segment"]*len(segments),segments))}
 
ManufOrTopC ="Top Companies"
BrandOrTopB="Brand"

 

In [3]:
# Generate the past 36 months
past_36_months = pd.date_range(end=end_date , periods=36, freq='M').strftime('%b-%y').tolist()
# Generate the past 12 months
past_12_months = pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
# Generate the past 3 months
past_3_months = pd.date_range(end=end_date , periods=3, freq='M').strftime('%b-%y').tolist()


#### Opening Excel

In [4]:
f_path = Path.cwd()
excel = client.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True  # False
wb = excel.Workbooks.Open(f_name)
ws=wb.Sheets([s.Name for s in wb.Sheets][0])
s_name = [s.Name for s in wb.Sheets][0]
pvtTable = ws.PivotTables(1)

#change report layout
pvtTable.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form

#change pivot table style
#Select from Design tab, try out Medium9 or Medium3
pvtTable.TableStyle2 = "pivotStyleMedium21"
pvtTable.ClearTable()

pvtTable.TableRange2.Cut(ws.Range("A16"))


fieldsNamePosition={}
for i in range(1,pvtTable.CubeFields.Count+1):
    fieldsNamePosition[str(pvtTable.CubeFields(i))]=i

In [5]:
# filter_dictionary_keys(fieldsNamePosition, "QuarterStart")

In [6]:
start_time = time.time()

## Price Positioning

In [105]:
price_positioning_brands = {}

In [106]:
row_list=[f'[Products].[{BrandOrTopB}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Relative Price]','[Measures].[Av Price/Unit]','[Measures].[Value Sales]','[Measures].[Value Share]','[Measures].[Value Share DYA]','[Measures].[Av Price/KG]','[Measures].[IYA Price/KG]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items


if len(categories)!=0:
    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_positioning_brands,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_positioning_brands, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_positioning_brands, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_positioning_brands, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_positioning_brands, '',subcategories,'[Products].[SubCategory].[SubCategory]')


with open("Pricing Datasets/price_positioning_brands.pickle", 'wb') as handle:
    pickle.dump(price_positioning_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)    

#### Price_Positioning Slide By Manufacturer

In [107]:
price_positioning_manuf= {}

In [108]:
row_list=[f'[Products].[{ManufOrTopC}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Relative Price]','[Measures].[Av Price/Unit]','[Measures].[Value Sales]','[Measures].[Value Share]','[Measures].[Value Share DYA]','[Measures].[Av Price/KG]','[Measures].[IYA Price/KG]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
if len(categories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_positioning_manuf,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_positioning_manuf, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_positioning_manuf, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_positioning_manuf, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_positioning_manuf, '',subcategories,'[Products].[SubCategory].[SubCategory]')
    
with open("Pricing Datasets/price_positioning_manuf.pickle", 'wb') as handle:
    pickle.dump(price_positioning_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)    

## Sector/Segment/SubSegment Leaderships (SINGLE)

In [109]:
sector_leadership = {}
segment_leadership = {}
subseg_leadership = {}
subcat_leadership = {}

sector_total_leadership = {}
segment_total_leadership = {}
subseg_total_leadership = {}
subcat_total_leadership = {}

if len(sectors) != 0:
    row_list=['[Products].[Sector]']  
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Value Sales]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Gross Margin %]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(sector_total_leadership,'')

    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
        calculate_data(sector_leadership,f'{brand} | ')


with open('Pricing Datasets/sector_total_leadership.pickle', 'wb') as handle:
    pickle.dump(sector_total_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Pricing Datasets/sector_leadership.pickle', 'wb') as handle:
    pickle.dump(sector_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)  


if len(segments) != 0:
    row_list=['[Products].[Segment]']  
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Value Sales]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Gross Margin %]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items



    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(segment_total_leadership,'')

    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
        calculate_data(segment_leadership,f'{brand} | ')


with open('Pricing Datasets/segment_total_leadership.pickle', 'wb') as handle:
    pickle.dump(segment_total_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Pricing Datasets/segment_leadership.pickle', 'wb') as handle:
    pickle.dump(segment_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)  



if len(subsegments) != 0:
    row_list=['[Products].[SubSegment]']  
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Value Sales]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Gross Margin %]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items



    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(subseg_total_leadership,'')

    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
        calculate_data(subseg_leadership,f'{brand} | ')


with open('Pricing Datasets/subseg_total_leadership.pickle', 'wb') as handle:
    pickle.dump(subseg_total_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Pricing Datasets/subseg_leadership.pickle', 'wb') as handle:
    pickle.dump(subseg_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)  




if len(subcategories) != 0:
    row_list=['[Products].[SubCategory]']  
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Value Sales]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Gross Margin %]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(subcat_total_leadership,'')

    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
        calculate_data(subcat_leadership,f'{brand} | ')


with open('Pricing Datasets/subcat_total_leadership.pickle', 'wb') as handle:
    pickle.dump(subcat_total_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Pricing Datasets/subcat_leadership.pickle', 'wb') as handle:
    pickle.dump(subcat_leadership, handle, protocol=pickle.HIGHEST_PROTOCOL)  


## Sector/Segment Leaderships

In [110]:
for key_child, val_child in direct_parent.items():
    if val_child[0][0] == "Category":
        continue  # Skip this iteration and continue to the next one

    print(key_child,[i[1] for i in val_child],val_child[0][0])
    # for key_par, val_par in val_child.items():/
    globals()[f"total_{key_child}"] = {}
    globals()[f"brand_{key_child}"] = {}
    globals()[f"manuf_{key_child}"]={}
    
    row_list=[f'[Products].[{val_child[0][0]}]',f'[Products].[{key_child}]']
    column_list=[]
    filter_list=[f'[Products].[{ManufOrTopC}]','[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Value Sales]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Gross Margin %]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(globals()[f"total_{key_child}"],'')

    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
        calculate_data(globals()[f"brand_{key_child}"],f'{brand} | ')

    if len(client_manuf)>1:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True    
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"


    calculate_category_data(globals()[f"manuf_{key_child}"],f' | {client_manuf[0]}')

    with open(f'Pricing Datasets/total_{key_child}.pickle', 'wb') as handle:
        pickle.dump(globals()[f"total_{key_child}"], handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'Pricing Datasets/brand_{key_child}.pickle', 'wb') as handle:
        pickle.dump(globals()[f"brand_{key_child}"], handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'Pricing Datasets/manuf_{key_child}.pickle', 'wb') as handle:
        pickle.dump(globals()[f"manuf_{key_child}"], handle, protocol=pickle.HIGHEST_PROTOCOL)

Segment ['System', 'Disposables'] Sector
SubSegment ['Razors', 'Refills', 'Disposables'] Segment
SubCategory ['Razors', 'Refills', 'Disposables'] Segment


## Shelf & Avg Price / KG By Brand

In [111]:
if len(sectors) != 0:
    row_list=[ '[Products].[Sector]',f'[Products].[{BrandOrTopB}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Relative Price]','[Measures].[Av Price/Unit]','[Measures].[Value Sales]','[Measures].[IYA Price/KG]','[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    shelf_sectors_all_brands = {}
    shelf_sectors_top_brands = {}
    ### The All brands (need only the Sectors Total rows)

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(shelf_sectors_all_brands,'')

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(shelf_sectors_top_brands,'',[],'')



    with open('Pricing Datasets/shelf_sectors_top_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_sectors_top_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('Pricing Datasets/shelf_sectors_all_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_sectors_all_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        

In [112]:
if len(segments) != 0:
    row_list=[ '[Products].[Segment]',f'[Products].[{BrandOrTopB}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Products].[Sector]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Relative Price]','[Measures].[Av Price/Unit]','[Measures].[Value Sales]','[Measures].[IYA Price/KG]','[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    shelf_segments_all_brands = {}
    shelf_segments_top_brands = {}

    ### The All brands (need only the Sectors Total rows)

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    # calculate_data(shelf_segments_all_brands,'',[],'')
    calculate_data(shelf_segments_all_brands, '',sectors,'[Products].[Sector].[Sector]')

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    # calculate_data(shelf_segments_top_brands,'',[],'')
    calculate_data(shelf_segments_top_brands, '',sectors,'[Products].[Sector].[Sector]')

        
    with open('Pricing Datasets/shelf_segments_top_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_segments_top_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

    with open('Pricing Datasets/shelf_segments_all_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_segments_all_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)
        


In [113]:
if len(subsegments)!=0:
    row_list=[ '[Products].[SubSegment]',f'[Products].[{BrandOrTopB}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Products].[Segment]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Relative Price]','[Measures].[Av Price/Unit]','[Measures].[Value Sales]','[Measures].[IYA Price/KG]','[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    shelf_subsegments_all_brands = {}
    shelf_subsegments_top_brands = {}

    ### The All brands (need only the Sectors Total rows)

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(shelf_subsegments_all_brands, '',segments,'[Products].[Segment].[Segment]')

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(shelf_subsegments_top_brands, '',segments,'[Products].[Segment].[Segment]')

    
    with open('Pricing Datasets/shelf_subsegments_top_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_subsegments_top_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

    with open('Pricing Datasets/shelf_subsegments_all_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_subsegments_all_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)
        


In [114]:
subcatg_parent = "Segment"
subcatg_parent_list = segments

In [115]:
if len(subcategories)!=0:
    row_list=[ '[Products].[SubCategory]',f'[Products].[{BrandOrTopB}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Products].[Segment]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Relative Price]','[Measures].[Av Price/Unit]','[Measures].[Value Sales]','[Measures].[IYA Price/KG]','[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    shelf_subcategories_all_brands = {}
    shelf_subcategories_top_brands = {}

    ### The All brands (need only the Sectors Total rows)

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(shelf_subcategories_all_brands, '',subcatg_parent_list,f'[Products].[{subcatg_parent}].[{subcatg_parent}]')

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[SubCategory]' 
    calculate_data(shelf_subcategories_top_brands, '',subcatg_parent_list,f'[Products].[{subcatg_parent}].[{subcatg_parent}]')

        
    with open('Pricing Datasets/shelf_subcategories_top_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_subcategories_top_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    with open('Pricing Datasets/shelf_subcategories_all_brands.pickle', 'wb') as handle:
        pickle.dump(shelf_subcategories_all_brands, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Shelf & Avg Price / KG By Manuf

In [116]:
if len(sectors) != 0:
    row_list=[ '[Products].[Sector]',f'[Products].[{ManufOrTopC}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    shelf_sectors_all_manuf = {}
    shelf_sectors_top_manuf= {}
    ### The All brands (need only the Sectors Total rows)

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(shelf_sectors_all_manuf,'')

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(shelf_sectors_top_manuf,'',[],'')



    with open('Pricing Datasets/shelf_sectors_top_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_sectors_top_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('Pricing Datasets/shelf_sectors_all_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_sectors_all_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        

In [117]:
if len(segments) != 0:
    row_list=[ '[Products].[Segment]',f'[Products].[{ManufOrTopC}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Products].[Sector]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    shelf_segments_all_manuf = {}
    shelf_segments_top_manuf = {}

    ### The All brands (need only the Sectors Total rows)

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    # calculate_data(shelf_segments_all_brands,'',[],'')
    calculate_data(shelf_segments_all_manuf, '',sectors,'[Products].[Sector].[Sector]')

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    # calculate_data(shelf_segments_top_brands,'',[],'')
    calculate_data(shelf_segments_top_manuf, '',sectors,'[Products].[Sector].[Sector]')

        
    with open('Pricing Datasets/shelf_segments_top_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_segments_top_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

    with open('Pricing Datasets/shelf_segments_all_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_segments_all_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)
        


In [118]:
if len(subsegments)!=0:
    row_list=[ '[Products].[SubSegment]',f'[Products].[{ManufOrTopC}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Products].[Segment]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
    shelf_subsegments_all_manuf = {}
    shelf_subsegments_top_manuf = {}
    ### The All brands (need only the Sectors Total rows)
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(shelf_subsegments_all_manuf, '',segments,'[Products].[Segment].[Segment]')
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(shelf_subsegments_top_manuf, '',segments,'[Products].[Segment].[Segment]')
    
    with open('Pricing Datasets/shelf_subsegments_top_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_subsegments_top_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('Pricing Datasets/shelf_subsegments_all_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_subsegments_all_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)
        


In [119]:
if len(subcategories)!=0:
    row_list=[ '[Products].[SubCategory]',f'[Products].[{ManufOrTopC}]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Products].[Segment]','[Products].[SubSegment]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Base Price/KG]','[Measures].[Av Price/KG]', '[Measures].[Value Share]','[Measures].[WoB %]','[Measures].[Value Share DYA]']
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
    shelf_subcategories_all_manuf = {}
    shelf_subcategories_top_manuf = {}
    ### The All brands (need only the Sectors Total rows)
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(shelf_subcategories_all_manuf, '',subcatg_parent_list,f'[Products].[{subcatg_parent}].[{subcatg_parent}]')
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[SubCategory]' 
    calculate_data(shelf_subcategories_top_manuf, '',subcatg_parent_list,f'[Products].[{subcatg_parent}].[{subcatg_parent}]')

    with open('Pricing Datasets/shelf_subcategories_top_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_subcategories_top_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('Pricing Datasets/shelf_subcategories_all_manuf.pickle', 'wb') as handle:
        pickle.dump(shelf_subcategories_all_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Price Point Distribution By Product (Item)


In [120]:
direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories)),
                "Segment":list(zip(["Sector"]*len(sectors),sectors)),
                "SubSegment":list(zip(["Segment"]*len(segments),segments)),
                "SubCategory":list(zip(["Segment"]*len(segments),segments))}
 

In [121]:
for ke, val in direct_parent.items():
    globals()[f"price_distribution_{ke}_p3m"] = {}
    globals()[f"price_distribution_{ke}_p12m"] = {}
    globals()[f"price_distribution_{ke}_manuf_p3m"] = {}
    globals()[f"price_distribution_{ke}_manuf_p12m"] = {}
    

    row_list=[f'[Products].[{ke}]',f'[Products].[{prodORitem}]','[Products].[Total Size]']
    column_list=[]
    filter_list=[f'[Products].[{ManufOrTopC}]','[Products].[Category]','[Scope].[Scope]',f'[Products].[{BrandOrTopB}]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{val[0][0]}]']
    value_list=['[Measures].[Base Price/Unit]','[Measures].[Base Price/KG]', '[Measures].[Value Sales]','[Measures].[Gross Margin %]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[{val[0][0]}]'

    if len(categories):
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()

    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
    calculate_data(globals()[f"price_distribution_{ke}_manuf_p12m"],f'{client_manuf[0]} | ')

    
    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'

        if val[0][0] =="Category":
            calculate_data(globals()[f"price_distribution_{ke}_p12m"], f'{categories[0]} | {brand} | ',[],'')

        else:
            calculate_data(globals()[f"price_distribution_{ke}_p12m"], f'{brand} | ',[i[1] for i in val],f'[Products].[{val[0][0]}].[{val[0][0]}]')

    ######################################################## P3M #################################################################
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_3_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    if len(client_manuf)>1:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
    calculate_data(globals()[f"price_distribution_{ke}_manuf_p3m"],f'{client_manuf[0]} | ')

    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
      
        if val[0][0] =="Category":
            calculate_data(globals()[f"price_distribution_{ke}_p3m"], f'{categories[0]} | {brand} | ',[],'')

        else:
            calculate_data(globals()[f"price_distribution_{ke}_p3m"], f'{brand} | ',[i[1] for i in val],f'[Products].[{val[0][0]}].[{val[0][0]}]')

    with open(f'Pricing Datasets/price_distribution_{ke}_p3m.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_{ke}_p3m"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'Pricing Datasets/price_distribution_{ke}_p12m.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_{ke}_p12m"], handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'Pricing Datasets/price_distribution_{ke}_manuf_p3m.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_{ke}_manuf_p3m"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'Pricing Datasets/price_distribution_{ke}_manuf_p12m.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_{ke}_manuf_p12m"], handle, protocol=pickle.HIGHEST_PROTOCOL)


## Price Point Distribution By Product Scraped(Item)

In [14]:
direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories)),
                "Segment":list(zip(["Sector"]*len(sectors),sectors)),
                "SubSegment":list(zip(["Segment"]*len(segments),segments)),
                "SubCategory":list(zip(["Segment"]*len(segments),segments))}

In [15]:
for ke, val in direct_parent.items():
    globals()[f"price_distribution_scraped_{ke}"] = {}
    globals()[f"price_distribution_scraped_{ke}_manuf"] = {}
    

    row_list=[]#'[Calendar].[End of Week]',f'[Products].[{ke}]',f'[Products].[{prodORitem}]',f'[Products].[Ean]','[Products].[Total Size]']
    column_list=[]
    filter_list=[f'[Products].[{ManufOrTopC}]','[Products].[Category]','[Scope].[Scope]',f'[Products].[{BrandOrTopB}]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{val[0][0]}]']
    value_list=['[Measures].[Scraped Av. Price/Unit]','[Measures].[Scraped Av. Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[{val[0][0]}]'

    if len(categories):
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    # visible_items = []
    # for value in past_12_months:
    #     visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()

    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
    row_list=['[Calendar].[End of Week]',f'[Products].[{ke}]',f'[Products].[{prodORitem}]',f'[Products].[Ean]','[Products].[Total Size]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_data(globals()[f"price_distribution_scraped_{ke}_manuf"],f'{client_manuf[0]} | ')

    
    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'

        if val[0][0] =="Category":
            calculate_data(globals()[f"price_distribution_scraped_{ke}"], f'{categories[0]} | {brand} | ',[],'')

        else:
            calculate_data(globals()[f"price_distribution_scraped_{ke}"], f'{brand} | ',[i[1] for i in val],f'[Products].[{val[0][0]}].[{val[0][0]}]')

    with open(f'Pricing Datasets/price_distribution_scraped_{ke}.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_scraped_{ke}"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    with open(f'Pricing Datasets/price_distribution_scraped_{ke}_manuf.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_scraped_{ke}_manuf"], handle, protocol=pickle.HIGHEST_PROTOCOL)


In [16]:
for ke, val in direct_parent.items():
    globals()[f"price_distribution_scraped_{ke}_Ean"] = {}
    globals()[f"price_distribution_scraped_{ke}_Ean_manuf"] = {}


    row_list=[]#f'[Products].[Ean]',f'[Products].[Value Sales]']
    column_list=[]
    filter_list=[f'[Products].[{ManufOrTopC}]','[Products].[Category]','[Scope].[Scope]',f'[Products].[{BrandOrTopB}]','[Market].[Area]','[Calendar].[MonthYear]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{val[0][0]}]']
    value_list=['[Measures].[Value Sales]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[{val[0][0]}]'

    if len(categories):
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()

    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
    row_list=['[Products].[Ean]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_data(globals()[f"price_distribution_scraped_{ke}_Ean_manuf"],f'{client_manuf[0]} | ')

    
    for brand in client_brands:
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'

        if val[0][0] =="Category":
            calculate_data(globals()[f"price_distribution_scraped_{ke}_Ean"], f'{categories[0]} | {brand} | ',[],'')

        else:
            calculate_data(globals()[f"price_distribution_scraped_{ke}_Ean"], f'{brand} | ',[i[1] for i in val],f'[Products].[{val[0][0]}].[{val[0][0]}]')

    with open(f'Pricing Datasets/price_distribution_scraped_{ke}_Ean.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_scraped_{ke}_Ean"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    with open(f'Pricing Datasets/price_distribution_scraped_{ke}_Ean_manuf.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_scraped_{ke}_Ean_manuf"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    

## Price Point Distribution By Brand

##### By Brand For CATEGORY (Slide 6)

In [7]:
price_distribution_by_brands_category = {}
row_list=[ f'[Products].[{BrandOrTopB}]','[Products].[Pack Size]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Av Price/Unit]', '[Measures].[Value Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) !=0:
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items


################################################## Scope = Category ######################################################
if len(categories) !=0:
  pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
  pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
  calculate_data(price_distribution_by_brands_category, f'{categories[0]} | ',[],'')


################################################## Scope = Sector ######################################################
if len(sectors) !=0:
  pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
  pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
  calculate_data(price_distribution_by_brands_category, '',sectors,'[Products].[Sector].[Sector]')

if len(segments) !=0:
  pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
  pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
  calculate_data(price_distribution_by_brands_category, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments) !=0:
  pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
  pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
  calculate_data(price_distribution_by_brands_category, '',subsegments,'[Products].[SubSegment].[SubSegment]')
  
if len(subcategories) !=0:
  pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
  pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
  calculate_data(price_distribution_by_brands_category, '',subcategories,'[Products].[SubCategory].[SubCategory]')

with open('Pricing Datasets/price_distribution_by_brands_category.pickle', 'wb') as handle:
    pickle.dump(price_distribution_by_brands_category, handle, protocol=pickle.HIGHEST_PROTOCOL)

##### By brand For Sec,Seg,SubSeg... (Slide 7)


In [8]:
for ke, val in direct_parent.items():
    globals()[f"price_distribution_by_brands_{ke}"] = {}

    row_list=[f'[Products].[{BrandOrTopB}]',f'[Products].[{ke}]','[Products].[Pack Size]']
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]', f'[Products].[{val[0][0]}]']
    value_list=['[Measures].[Av Price/Unit]', '[Measures].[Value Share]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
    if val[0][0] == "Category":
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_data(globals()[f"price_distribution_by_brands_{ke}"],f'{categories[0]} | ',[],'')

    else:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[{val[0][0]}]'
        calculate_data(globals()[f"price_distribution_by_brands_{ke}"], '', [i[1] for i in val], f'[Products].[{val[0][0]}].[{val[0][0]}]')


    with open(f'Pricing Datasets/price_distribution_by_brands_{ke}.pickle', 'wb') as handle:
        pickle.dump(globals()[f"price_distribution_by_brands_{ke}"], handle, protocol=pickle.HIGHEST_PROTOCOL)

## Price Point Comparison by Product

In [8]:
price_point_by_brands_items_P3M = {}
price_point_by_brands_items_P12M = {}

In [9]:
row_list=[ f'[Products].[{BrandOrTopB}]',f'[Products].[{prodORitem}]','[Products].[Total Size]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Base Price/Unit]', '[Measures].[Base Price/KG]' ,'[Measures].[Value Sales]','[Measures].[Value Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_3_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

In [10]:
if len(categories) !=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_point_by_brands_items_P3M,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_point_by_brands_items_P3M, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_point_by_brands_items_P3M, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_point_by_brands_items_P3M, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_point_by_brands_items_P3M, '',subcategories,'[Products].[SubCategory].[SubCategory]')

##### By P12M To Get the Gross Margin%

In [11]:
row_list=[ f'[Products].[{BrandOrTopB}]',f'[Products].[{prodORitem}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Gross Margin %]','[Measures].[Value Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
if len(categories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_point_by_brands_items_P12M,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_point_by_brands_items_P12M, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_point_by_brands_items_P12M, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_point_by_brands_items_P12M, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_point_by_brands_items_P12M, '',subcategories,'[Products].[SubCategory].[SubCategory]')

In [13]:
with open('Pricing Datasets/price_point_by_brands_items_P3M.pickle', 'wb') as handle:
    pickle.dump(price_point_by_brands_items_P3M, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Pricing Datasets/price_point_by_brands_items_P12M.pickle', 'wb') as handle:
     pickle.dump(price_point_by_brands_items_P12M, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Price Point Comparison by Product (scraped)

In [7]:
price_point_by_brands_items_scraped = {}

In [8]:
row_list=[]#'[Calendar].[End of Week]', f'[Products].[{BrandOrTopB}]',f'[Products].[{prodORitem}]','[Products].[Total Size]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Scraped Av. Price/Unit]','[Measures].[Scraped Av. Price/KG]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
row_list=['[Calendar].[End of Week]', f'[Products].[{BrandOrTopB}]',f'[Products].[{prodORitem}]','[Products].[Ean]','[Products].[Total Size]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
# pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True
visible_items = []
# for value in past_3_months:
#     visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

In [9]:
if len(categories) !=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_point_by_brands_items_scraped,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_point_by_brands_items_scraped, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_point_by_brands_items_scraped, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_point_by_brands_items_scraped, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_point_by_brands_items_scraped, '',subcategories,'[Products].[SubCategory].[SubCategory]')

In [10]:
with open('Pricing Datasets/price_point_by_brands_items_scraped.pickle', 'wb') as handle:
    pickle.dump(price_point_by_brands_items_scraped, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Price Point Comparison by Product by manuf

In [ ]:
price_point_by_manuf_items_P3M = {}
price_point_by_manuf_items_P12M = {}
row_list=[ f'[Products].[{ManufOrTopC}]',f'[Products].[{prodORitem}]','[Products].[Total Size]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Base Price/Unit]', '[Measures].[Base Price/KG]' ,'[Measures].[Value Sales]','[Measures].[Value Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_3_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

In [ ]:
if len(categories) !=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_point_by_manuf_items_P3M,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_point_by_manuf_items_P3M, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_point_by_manuf_items_P3M, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_point_by_manuf_items_P3M, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_point_by_manuf_items_P3M, '',subcategories,'[Products].[SubCategory].[SubCategory]')

In [ ]:
row_list=[f'[Products].[{ManufOrTopC}]',f'[Products].[{prodORitem}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Gross Margin %]','[Measures].[Value Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
if len(categories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_point_by_manuf_items_P12M,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_point_by_manuf_items_P12M, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_point_by_manuf_items_P12M, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_point_by_manuf_items_P12M, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_point_by_manuf_items_P12M, '',subcategories,'[Products].[SubCategory].[SubCategory]')

In [ ]:
with open('Pricing Datasets/price_point_by_manuf_items_P3M.pickle', 'wb') as handle:
    pickle.dump(price_point_by_manuf_items_P3M, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Pricing Datasets/price_point_by_manuf_items_P12M.pickle', 'wb') as handle:
     pickle.dump(price_point_by_manuf_items_P12M, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Price Point Comparison by Product by manuf (Scraped)

In [11]:
price_point_by_manuf_items_scraped = {}
row_list=[]#'[Calendar].[End of Week]', f'[Products].[{ManufOrTopC}]',f'[Products].[{prodORitem}]','[Products].[Ean]','[Products].[Total Size]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Scraped Av. Price/Unit]','[Measures].[Scraped Av. Price/KG]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

row_list=['[Calendar].[End of Week]', f'[Products].[{ManufOrTopC}]',f'[Products].[{prodORitem}]','[Products].[Ean]','[Products].[Total Size]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
# pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
# visible_items = []
# for value in past_3_months:
#     visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
# pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

In [12]:
if len(categories) !=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_point_by_manuf_items_scraped,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_point_by_manuf_items_scraped, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_point_by_manuf_items_scraped, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_point_by_manuf_items_scraped, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_point_by_manuf_items_scraped, '',subcategories,'[Products].[SubCategory].[SubCategory]')

In [13]:
with open('Pricing Datasets/price_point_by_manuf_items_scraped.pickle', 'wb') as handle:
    pickle.dump(price_point_by_manuf_items_scraped, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Price Correlation

In [25]:
price_correlation_P3Y = {}

#### For Past 3 Years

In [26]:
row_list=[ f'[Products].[{BrandOrTopB}]','[Calendar].[QuarterStart]', '[Calendar].[End of Week]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Volume Share]', '[Measures].[Av Price/KG]','[Measures].[Value Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_36_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

In [27]:
if len(categories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(price_correlation_P3Y,f'{categories[0]} | ')
if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(price_correlation_P3Y, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(price_correlation_P3Y, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(price_correlation_P3Y, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(price_correlation_P3Y, '',subcategories,'[Products].[SubCategory].[SubCategory]')

In [28]:
with open('Pricing Datasets/price_correlation_P3Y.pickle', 'wb') as handle:
    pickle.dump(price_correlation_P3Y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Wed Jul 23 15:10:21 2025
Script ended at: Wed Jul 23 15:52:13 2025
Elapsed time: 2512.61 seconds
